In [ ]:
!pip install "transformers==4.40.1" utilsforecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
from transformers import AutoModelForCausalLM

In [ ]:
# Initialize model


In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

## Zero-shot forecasting

In [ ]:
HORIZON = 13

In [ ]:
unique_stores = sorted(df['Store'].unique())
inputs_list = []

for store_id in unique_stores:
    store_df = df[df['Store'] == store_id].sort_values(by='Date')
    inputs_list.append(store_df['Weekly_Sales'].values)

# Create tensor from input list

print(input_tensor.shape)

In [ ]:
# Make predictions

print(output.shape)

In [ ]:
# Create DataFrame from Sundial's output
def create_forecast_df(
    quantile_forecast_tensor: torch.Tensor,
    original_df: pd.DataFrame,
    id_col: str,
    time_col: str,
    target_col: str,
    horizon: int,
    freq: str,
):
    # Calculate quantiles across the num_samples dimension (dim=1)
    median_forecast = torch.quantile(quantile_forecast_tensor, 0.5, dim=1).numpy() # median
    lo_80_forecast = torch.quantile(quantile_forecast_tensor, 0.1, dim=1).numpy()  # 10th percentile
    hi_80_forecast = torch.quantile(quantile_forecast_tensor, 0.9, dim=1).numpy()  # 90th percentile

    # Get num_series and horizon

    # List for all rows of predictions

    # Sort all unique ids


    for i, id in enumerate(unique_ids):
        # Get the last known date for this id from the original DataFrame

        # Generate forecast dates

        for h in range(horizon):
            forecast_row = {
                id_col: id,
                time_col: forecast_dates[h],
                'sundial': median_forecast[i, h],
                'sundial-lo-80': lo_80_forecast[i, h],
                'sundial-hi-80': hi_80_forecast[i, h]
            }
            # Append all rows


    return pd.DataFrame(all_forecast_rows)


In [ ]:
# Create forecast DataFrame

fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
# Cross-validation with Sundial
def sundial_cv(
    df: pd.DataFrame,
    model,
    horizon: int,
    n_windows: int,
    id_col: str,
    time_col: str,
    target_col: str,
    freq: str,
):

    # List for all results

    # Max date

    for i in range(n_windows):
        # Calculate the cutoff date for the current window

        # Create a training DataFrame up to the cutoff_date

        # Prepare inputs_list


        # Generate forecasts


        # Convert forecasts to DataFrame
        fcsts_df = create_forecast_df(
            quantile_forecast_tensor=output,
            original_df=df_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            horizon=horizon,
            freq=freq,
        )

        # Add cutoff column

        # Append all results
        all_cv_forecasts.append(fcsts_df)

    # Concatenate results

    # Merge actual values


    return cv_df

In [ ]:
# Run cross-validation

cv_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df.drop(columns=["cutoff", "Weekly_Sales"]),
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
eval_df = evaluate(
    cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    models=['sundial'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df